In [1]:
import sys, os
sys.path.append('..')
from utils_ft import get_model_path, get_output_dir, get_root_path, to_relative
from scripts.train_mlx import run_mlx_training, fuse_model

# Config
MODEL_TYPE = "qwen"
#MODEL_TYPE = "llama" 
base_model = get_model_path(MODEL_TYPE)
data_folder = os.path.join(get_root_path(), "Finetuning", "data")
adapter_path = os.path.join(get_root_path(), "Finetuning", "adapters", MODEL_TYPE + "_mlx")
os.makedirs(adapter_path, exist_ok=True)
fused_output = get_output_dir(MODEL_TYPE, "mlx")

In [2]:
run_mlx_training(base_model, data_folder, adapter_path, iters=300)

--- Initializing MLX Fine-tuning (CLI Mode) ---
--- Model: ./local_models/Qwen/Qwen2.5-7B-Instruct-bfloat16 ---
--- Data Folder: ./Finetuning/data ---
Generating configuration file at: ./Finetuning/adapters/qwen_mlx/train_config.yaml
Executing MLX CLI training...
Loading configuration file /Users/arjandeweerd/AI_Projects/Ass3_LLM_ASP/SchASPLM-Shamantics/Finetuning/adapters/qwen_mlx/train_config.yaml
Loading pretrained model


The tokenizer you are loading from '/Users/arjandeweerd/AI_Projects/Ass3_LLM_ASP/SchASPLM-Shamantics/local_models/Qwen/Qwen2.5-7B-Instruct-bfloat16' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


Loading datasets
Training
Trainable parameters: 0.303% (23.069M/7615.617M)
Starting training..., iters: 300


Calculating loss...: 100%|██████████| 5/5 [00:06<00:00,  1.38s/it]


Iter 1: Val loss 1.241, Val took 6.897s
Iter 10: Train loss 1.298, Learning Rate 5.000e-05, It/sec 0.404, Tokens/sec 321.968, Trained Tokens 7960, Peak mem 18.454 GB


...

Iter 260: Val loss 0.771, Val took 5.301s
Iter 260: Train loss 0.032, Learning Rate 4.459e-05, It/sec 0.410, Tokens/sec 328.783, Trained Tokens 207579, Peak mem 18.547 GB
Iter 270: Train loss 0.015, Learning Rate 4.426e-05, It/sec 0.406, Tokens/sec 326.025, Trained Tokens 215610, Peak mem 18.547 GB


Calculating loss...: 100%|██████████| 5/5 [00:03<00:00,  1.50it/s]


Iter 280: Val loss 0.359, Val took 3.331s
Iter 280: Train loss 0.017, Learning Rate 4.375e-05, It/sec 0.460, Tokens/sec 324.649, Trained Tokens 222664, Peak mem 18.547 GB
Iter 290: Train loss 0.016, Learning Rate 4.340e-05, It/sec 0.487, Tokens/sec 327.317, Trained Tokens 229385, Peak mem 18.547 GB


Calculating loss...: 100%|██████████| 5/5 [00:05<00:00,  1.03s/it]


Iter 300: Val loss 0.880, Val took 5.156s
Iter 300: Train loss 0.019, Learning Rate 4.286e-05, It/sec 0.392, Tokens/sec 329.767, Trained Tokens 237795, Peak mem 18.547 GB
Iter 300: Saved adapter weights to /Users/arjandeweerd/AI_Projects/Ass3_LLM_ASP/SchASPLM-Shamantics/Finetuning/adapters/qwen_mlx/adapters.safetensors and /Users/arjandeweerd/AI_Projects/Ass3_LLM_ASP/SchASPLM-Shamantics/Finetuning/adapters/qwen_mlx/0000300_adapters.safetensors.
Saved final weights to /Users/arjandeweerd/AI_Projects/Ass3_LLM_ASP/SchASPLM-Shamantics/Finetuning/adapters/qwen_mlx/adapters.safetensors.
✓ Training completed successfully. Adapters saved.


In [3]:
fuse_model(base_model, adapter_path, fused_output)
print(f"Success! Model ready at {fused_output}")

Fusing LoRA adapters into base model (CLI Mode)...
--- Base Model: ./local_models/Qwen/Qwen2.5-7B-Instruct-bfloat16 ---
--- Adapter Path: ./Finetuning/adapters/qwen_mlx ---
--- Save Path: ./local_models/ft_qwen_mlx ---
Loading pretrained model


The tokenizer you are loading from '/Users/arjandeweerd/AI_Projects/Ass3_LLM_ASP/SchASPLM-Shamantics/local_models/Qwen/Qwen2.5-7B-Instruct-bfloat16' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


Success! Fused model saved to: ./local_models/ft_qwen_mlx
Success! Model ready at /Users/arjandeweerd/AI_Projects/Ass3_LLM_ASP/SchASPLM-Shamantics/local_models/ft_qwen_mlx
